In [ ]:
import torch
torch.cuda.is_available()


In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git


In [ ]:
import torch
import clip
import numpy as np
from PIL import Image
import os
import shutil
import matplotlib.pyplot as plt


In [ ]:
os.makedirs("/content/shirts", exist_ok=True)


In [ ]:
from google.colab import files
files.upload()


In [ ]:
for file in os.listdir("/content"):
    if file.lower().endswith((".jpg", ".png", ".jpeg")):
        shutil.move(f"/content/{file}", f"/content/shirts/{file}")

os.listdir("/content/shirts")


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
print("CLIP loaded on:", device)


In [ ]:
image_embeddings_list = []
image_names = []

for img_name in os.listdir("/content/shirts"):
    img_path = f"/content/shirts/{img_name}"
    image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)

    with torch.no_grad():
        emb = model.encode_image(image)
        emb = emb / emb.norm(dim=-1, keepdim=True)

    image_embeddings_list.append(emb.cpu().numpy())
    image_names.append(img_name)

image_embeddings = np.vstack(image_embeddings_list)

print("Total images:", len(image_names))
print("Embedding shape:", image_embeddings.shape)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

query_index = 0

query_emb = image_embeddings[query_index].reshape(1, -1)
scores = cosine_similarity(query_emb, image_embeddings)[0]

top_indices = scores.argsort()[::-1][1:6]

print("Query:", image_names[query_index])
for i in top_indices:
    print(image_names[i], round(scores[i], 3))


In [ ]:
plt.figure(figsize=(15,4))

plt.subplot(1,6,1)
plt.imshow(Image.open(f"/content/shirts/{image_names[query_index]}"))
plt.title("Query")
plt.axis("off")

for i, idx in enumerate(top_indices):
    plt.subplot(1,6,i+2)
    plt.imshow(Image.open(f"/content/shirts/{image_names[idx]}"))
    plt.title(f"Rec {i+1}")
    plt.axis("off")

plt.show()


In [ ]:
style_texts = [
    "minimal casual outfit for everyday wear",
    "smart casual shirt suitable for office",
    "streetwear fashion with layered clothing",
    "formal outfit for meetings or events",
    "relaxed home or lounge wear"
]


In [ ]:
text_tokens = clip.tokenize(style_texts).to(device)

with torch.no_grad():
    text_embeddings = model.encode_text(text_tokens)
    text_embeddings = text_embeddings / text_embeddings.norm(dim=-1, keepdim=True)

print(text_embeddings.shape)


In [ ]:
import torch

# convert numpy → torch tensor
img_emb = torch.tensor(
    image_embeddings[query_index],
    device=device
).unsqueeze(0)

# similarity between image and style texts
scores = (img_emb @ text_embeddings.T).squeeze(0)

best_style_idx = scores.argmax().item()

print("Predicted style:", style_texts[best_style_idx])


In [ ]:
style_advice = {
    "minimal casual outfit for everyday wear":
        "This shirt fits a clean, minimal casual style. Pair it with jeans or chinos and white sneakers.",

    "smart casual shirt suitable for office":
        "This shirt suits a smart-casual office look. Combine it with dark trousers and loafers.",

    "streetwear fashion with layered clothing":
        "This shirt works well for a streetwear vibe. Try layering it with a jacket or hoodie and sneakers.",

    "formal outfit for meetings or events":
        "This shirt can be styled formally. Wear it with tailored pants and formal shoes.",

    "relaxed home or lounge wear":
        "This shirt is ideal for relaxed wear. Pair it with joggers or shorts for comfort."
}

print("🧠 AI Shopping Buddy says:")
print(style_advice[style_texts[best_style_idx]])
